# Alzheimer's Disease and Frontotemporal Dementia Prediction


- Author: Elmo Chavez
- Date: 19-Jul-2023

**Description**

> [...]


## Read the Data


### Import Libraries


In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb

### Reading Datasets


In [2]:
# Filenames
#filename_participants = 'participants_selected.csv'
filename_epochs_fp1 = 'epoch_psd_features.csv'
filename_epochs_all = 'epoch_psd_features_all_channels.csv'
filename_epochs_bnds_fp1 = 'epoch_bands_psd_features.csv'
filename_epochs_bdns_all = 'epoch_bands_psd_features_all_channels.csv'

# path
path = os.getcwd()

# Participants info
#df_participants = pd.read_csv(path+'/Feature_Extraction/'+filename_participants)

# Epochs based feature extraction
df_epochs_fp1 = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_fp1)
df_epochs_all = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_all)

# Epochs and Frequency Bands feature extraction
df_epochs_bands_fp1 = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_bnds_fp1)
df_epochs_bands_all = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_bdns_all)

# Shape of the data
#print('Participants info:',df_participants.shape)
print('Epochs base Feature Extraction - FP1:',df_epochs_fp1.shape)
print('Epochs base Feature Extraction - All Channels:',df_epochs_all.shape)
print('Epochs base with Freq. Bands Feature Extraction - FP1:',df_epochs_bands_fp1.shape)
print('Epochs base with Freq. Bands Feature Extraction - All Channels:',df_epochs_bands_all.shape)

Epochs base Feature Extraction - FP1: (714, 11)
Epochs base Feature Extraction - All Channels: (714, 11)
Epochs base with Freq. Bands Feature Extraction - FP1: (714, 31)
Epochs base with Freq. Bands Feature Extraction - All Channels: (714, 31)


## Train Test


#### Epoch based FP1


In [3]:
df_epochs_fp1.head()

,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78,1,0,19.0,8.138737e-07,3.255495e-08,0.480204,1.953125,0.000013,0
1,sub-002,78,1,1,19.0,6.457802e-07,2.583121e-08,0.373726,1.953125,0.000010,0
2,sub-002,78,1,2,19.0,7.862074e-07,3.144830e-08,0.397717,1.953125,0.000013,0
3,sub-002,78,1,3,19.0,8.496011e-07,3.398405e-08,0.458611,1.953125,0.000013,0
4,sub-002,78,1,4,19.0,7.503180e-07,3.001272e-08,0.402417,1.953125,0.000012,0


In [4]:
df_epochs_fp1_grp = df_epochs_fp1.groupby('subject_id').mean().reset_index()
print(df_epochs_fp1_grp.shape)
df_epochs_fp1_grp.head()

(51, 11)


,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78.0,1.0,6.5,19.0,8.747903e-07,3.499161e-08,0.440120,1.953125,0.000014,0.0
1,sub-004,67.0,1.0,6.5,19.0,1.032087e-06,4.128349e-08,0.459969,1.953125,0.000016,0.0
2,sub-005,70.0,0.0,6.5,19.0,1.147346e-06,4.589383e-08,0.461923,1.953125,0.000017,0.0
3,sub-006,61.0,1.0,6.5,19.0,1.349809e-06,5.399235e-08,0.489254,1.953125,0.000020,0.0
4,sub-007,79.0,1.0,6.5,19.0,9.680973e-07,3.872389e-08,0.437120,1.953125,0.000015,0.0


In [5]:
cols_to_drop = ['subject_id','group']

X_epochs_fp1 = df_epochs_fp1_grp.drop(columns=cols_to_drop, axis=1).values
y_epochs_fp1 = df_epochs_fp1_grp['group'].values

#### Epochs based All Channels


In [6]:
df_epochs_all.head()

,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78,1,0,19.0,8.346040e-07,3.338416e-08,0.478456,1.953125,0.000013,0
1,sub-002,78,1,1,19.0,6.761889e-07,2.704755e-08,0.374997,1.953125,0.000011,0
2,sub-002,78,1,2,19.0,8.086165e-07,3.234466e-08,0.394570,1.953125,0.000013,0
3,sub-002,78,1,3,19.0,8.440792e-07,3.376317e-08,0.447835,1.953125,0.000013,0
4,sub-002,78,1,4,19.0,7.673421e-07,3.069369e-08,0.416419,1.953125,0.000012,0


In [7]:
df_epochs_all_grp = df_epochs_fp1.groupby('subject_id').mean().reset_index()

X_epochs_all = df_epochs_all_grp.drop(columns=cols_to_drop, axis=1).values
y_epochs_all = df_epochs_all_grp['group'].values

#### Epochs based with Frequency Bands FP1


In [8]:
df_epochs_bands_fp1.head()

,epoch_id,total_channels,delta_total_power,delta_avg_power,delta_relative_power,delta_peak_freq,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,...,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,subject_id,age,gender,group
0,0,19.0,5.356440e-07,2.678220e-07,0.316042,1.953125,0.000008,1.703684e-08,8.518421e-09,0.010052,...,0.000001,1.292570e-07,1.615713e-08,0.076265,44.921875,0.000002,sub-002,78,1,0
1,1,19.0,3.903346e-07,1.951673e-07,0.225894,1.953125,0.000006,2.239278e-08,1.119639e-08,0.012959,...,0.000002,1.108229e-07,1.385286e-08,0.064135,35.156250,0.000002,sub-002,78,1,0
2,2,19.0,5.090068e-07,2.545034e-07,0.257490,1.953125,0.000008,3.471327e-08,1.735663e-08,0.017560,...,0.000002,8.779028e-08,1.097378e-08,0.044410,37.109375,0.000002,sub-002,78,1,0
3,3,19.0,5.687169e-07,2.843584e-07,0.306991,1.953125,0.000008,2.759734e-08,1.379867e-08,0.014897,...,0.000001,1.364043e-07,1.705053e-08,0.073631,39.062500,0.000002,sub-002,78,1,0
4,4,19.0,5.352662e-07,2.676331e-07,0.287079,1.953125,0.000008,2.820804e-08,1.410402e-08,0.015129,...,0.000001,9.871103e-08,1.233888e-08,0.052942,42.968750,0.000002,sub-002,78,1,0


In [9]:
df_epochs_bands_fp1_grp = df_epochs_bands_fp1.groupby('subject_id').mean().reset_index()

X_epochs_bands_fp1 = df_epochs_bands_fp1_grp.drop(columns=cols_to_drop, axis=1).values
y_epochs_bands_fp1 = df_epochs_bands_fp1_grp['group'].values

#### Epochs based with Frequency Bands All Channels


In [10]:
df_epochs_bands_all.head()

,epoch_id,total_channels,delta_total_power,delta_avg_power,delta_relative_power,delta_peak_freq,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,...,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,subject_id,age,gender,group
0,0,19.0,5.607184e-07,2.803592e-07,0.321419,1.953125,0.000008,1.780230e-08,8.901150e-09,0.010167,...,0.000001,1.402790e-07,1.753488e-08,0.080389,41.118421,0.000002,sub-002,78,1,0
1,1,19.0,4.036695e-07,2.018348e-07,0.224124,1.953125,0.000006,2.129087e-08,1.064544e-08,0.011726,...,0.000002,1.166759e-07,1.458449e-08,0.064537,35.361842,0.000002,sub-002,78,1,0
2,2,19.0,5.249767e-07,2.624883e-07,0.256317,1.953125,0.000008,3.140269e-08,1.570135e-08,0.015305,...,0.000002,1.045114e-07,1.306392e-08,0.050853,37.109375,0.000002,sub-002,78,1,0
3,3,19.0,5.627172e-07,2.813586e-07,0.298903,1.953125,0.000008,2.816719e-08,1.408359e-08,0.014952,...,0.000002,1.184824e-07,1.481030e-08,0.062715,37.828947,0.000002,sub-002,78,1,0
4,4,19.0,5.368642e-07,2.684321e-07,0.291363,1.953125,0.000008,3.484860e-08,1.742430e-08,0.018918,...,0.000001,1.033295e-07,1.291619e-08,0.056063,39.268092,0.000002,sub-002,78,1,0


In [11]:
df_epochs_bands_all_grp = df_epochs_bands_all.groupby('subject_id').mean().reset_index()

X_epochs_bands_all = df_epochs_bands_all_grp.drop(columns=cols_to_drop, axis=1).values
y_epochs_bands_all = df_epochs_bands_all_grp['group'].values

## Predictions


In [12]:
cols_results = ['Feature Extraction', 'Description', 'Channels', 'Metrics', 'CV', 'ML Model', 'Accuracy']

df_results = pd.DataFrame(columns=cols_results)
df_results.columns

def append_result(result):
    index = len(df_results)
    df_results.loc[index] = result

### Epochs Based - FP1 Channel


In [13]:
# Variables for Results
approach = 'Time-Frequency Domain'
description = 'PSD Metrics from Epochs (1-50 Hz)'
channels = 'FP1'
metrics = 'Total Power, Relative Power, Peak Frequency, Spectral Entropy'
CV = 'KFold: 5 Folds'

# Define the classifiers
svm_classifier = SVC()
random_forest_classifier = RandomForestClassifier()
adaboost_classifier = AdaBoostClassifier()
xgboost_classifier = xgb.XGBClassifier()
lightgbm_classifier = lgb.LGBMClassifier()

# Create a KFold cross-validator with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

classifiers = [svm_classifier, random_forest_classifier, adaboost_classifier, xgboost_classifier, lightgbm_classifier]
classifier_names = ['SVM', 'Random Forest', 'AdaBoost', 'XGBoost', 'LightGBM']

for clf, clf_name in zip(classifiers, classifier_names):
    scores = []
    for train_index, test_index in kf.split(X_epochs_fp1):
        # Split the data into training and testing sets based on the current fold indices
        X_train, X_test = X_epochs_fp1[train_index], X_epochs_fp1[test_index]
        y_train, y_test = y_epochs_fp1[train_index], y_epochs_fp1[test_index]
        
        # Train the classifier on the training data
        clf.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = clf.predict(X_test)
        
        # Calculate accuracy for this fold and append to the scores list
        accuracy = np.mean(y_pred == y_test)
        scores.append(accuracy)
    
    #save results
    result = [approach, description, channels, metrics, CV, clf_name, np.mean(scores)]
    append_result(result)
    
    # Calculate and print the average accuracy across all folds
    print(f"{clf_name} - Average Accuracy: {round(np.mean(scores),4)}")
    
df_results.head()

SVM - Average Accuracy: 0.6455
Random Forest - Average Accuracy: 0.5091
AdaBoost - Average Accuracy: 0.3545
XGBoost - Average Accuracy: 0.3764
LightGBM - Average Accuracy: 0.5855


,Feature Extraction,Description,Channels,Metrics,CV,ML Model,Accuracy
0,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
1,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.509091
2,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
3,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
4,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455


### Epochs Based - All Channels


In [14]:
# Variables for Results
approach = 'Time-Frequency Domain'
description = 'PSD Metrics from Epochs (1-50 Hz)'
channels = 'All'
metrics = 'Total Power, Relative Power, Peak Frequency, Spectral Entropy'
CV = 'KFold: 5 Folds'

# Define the classifiers
svm_classifier2 = SVC()
random_forest_classifier2 = RandomForestClassifier()
adaboost_classifier2 = AdaBoostClassifier()
xgboost_classifier2 = xgb.XGBClassifier()
lightgbm_classifier2 = lgb.LGBMClassifier()

# Create a KFold cross-validator with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

classifiers2 = [svm_classifier2, random_forest_classifier2, adaboost_classifier2, xgboost_classifier2, lightgbm_classifier2]
classifier_names = ['SVM', 'Random Forest', 'AdaBoost', 'XGBoost', 'LightGBM']

for clf, clf_name in zip(classifiers2, classifier_names):
    scores = []
    for train_index, test_index in kf.split(X_epochs_all):
        # Split the data into training and testing sets based on the current fold indices
        X_train, X_test = X_epochs_all[train_index], X_epochs_all[test_index]
        y_train, y_test = y_epochs_all[train_index], y_epochs_all[test_index]
        
        # Train the classifier on the training data
        clf.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = clf.predict(X_test)
        
        # Calculate accuracy for this fold and append to the scores list
        accuracy = np.mean(y_pred == y_test)
        scores.append(accuracy)
        
    #save results
    result = [approach, description, channels, metrics, CV, clf_name, np.mean(scores)]
    append_result(result)
    
    # Calculate and print the average accuracy across all folds
    print(f"{clf_name} - Average Accuracy: {round(np.mean(scores),4)}")

df_results

SVM - Average Accuracy: 0.6455
Random Forest - Average Accuracy: 0.4727
AdaBoost - Average Accuracy: 0.3545
XGBoost - Average Accuracy: 0.3764
LightGBM - Average Accuracy: 0.5855


,Feature Extraction,Description,Channels,Metrics,CV,ML Model,Accuracy
0,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
1,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.509091
2,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
3,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
4,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455
5,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
6,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.472727
7,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
8,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
9,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455


### Epochs Based with Frequency Bands - FP1 Channel


In [15]:
# Variables for Results
approach = 'Time-Frequency Domain'
description = 'PSD Metrics from Epochs For each Frequency Band (Delta, Theta, Alpha, Beta, Gamma)'
channels = 'FP1'
metrics = 'Total Power, Relative Power, Peak Frequency, Spectral Entropy'
CV = 'KFold: 5 Folds'

# Define the classifiers
svm_classifier3 = SVC()
random_forest_classifier3 = RandomForestClassifier()
adaboost_classifier3 = AdaBoostClassifier()
xgboost_classifier3 = xgb.XGBClassifier()
lightgbm_classifier3 = lgb.LGBMClassifier()

# Create a KFold cross-validator with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

classifiers3 = [svm_classifier3, random_forest_classifier3, adaboost_classifier3, xgboost_classifier3, lightgbm_classifier3]
classifier_name = ['SVM', 'Random Forest', 'AdaBoost', 'XGBoost', 'LightGBM']

for clf, clf_name in zip(classifiers3, classifier_names):
    scores = []
    for train_index, test_index in kf.split(X_epochs_bands_fp1):
        # Split the data into training and testing sets based on the current fold indices
        X_train, X_test = X_epochs_bands_fp1[train_index], X_epochs_bands_fp1[test_index]
        y_train, y_test = y_epochs_bands_fp1[train_index], y_epochs_bands_fp1[test_index]
        
        # Train the classifier on the training data
        clf.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = clf.predict(X_test)
        
        # Calculate accuracy for this fold and append to the scores list
        accuracy = np.mean(y_pred == y_test)
        scores.append(accuracy)
    
    #save results
    result = [approach, description, channels, metrics, CV, clf_name, np.mean(scores)]
    append_result(result)
    
    # Calculate and print the average accuracy across all folds
    print(f"{clf_name} - Average Accuracy: {round(np.mean(scores),4)}")

df_results

SVM - Average Accuracy: 0.6455
Random Forest - Average Accuracy: 0.5709
AdaBoost - Average Accuracy: 0.4909
XGBoost - Average Accuracy: 0.5673
LightGBM - Average Accuracy: 0.5255


,Feature Extraction,Description,Channels,Metrics,CV,ML Model,Accuracy
0,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
1,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.509091
2,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
3,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
4,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455
5,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
6,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.472727
7,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
8,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
9,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455


### Epochs Based with Frequency Bands - All Channel


In [16]:
# Variables for Results
approach = 'Time-Frequency Domain'
description = 'PSD Metrics from Epochs For each Frequency Band (Delta, Theta, Alpha, Beta, Gamma)'
channels = 'All'
metrics = 'Total Power, Relative Power, Peak Frequency, Spectral Entropy'
CV = 'KFold: 5 Folds'

# Define the classifiers
svm_classifier4 = SVC()
random_forest_classifier4 = RandomForestClassifier()
adaboost_classifier4 = AdaBoostClassifier()
xgboost_classifier4 = xgb.XGBClassifier()
lightgbm_classifier4 = lgb.LGBMClassifier()

# Create a KFold cross-validator with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

classifiers4 = [svm_classifier4, random_forest_classifier4, adaboost_classifier4, xgboost_classifier4, lightgbm_classifier4]
classifier_name = ['SVM', 'Random Forest', 'AdaBoost', 'XGBoost', 'LightGBM']

for clf, clf_name in zip(classifiers4, classifier_names):
    scores = []
    for train_index, test_index in kf.split(X_epochs_bands_all):
        # Split the data into training and testing sets based on the current fold indices
        X_train, X_test = X_epochs_bands_all[train_index], X_epochs_bands_all[test_index]
        y_train, y_test = y_epochs_bands_all[train_index], y_epochs_bands_all[test_index]
        
        # Train the classifier on the training data
        clf.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = clf.predict(X_test)
        
        # Calculate accuracy for this fold and append to the scores list
        accuracy = np.mean(y_pred == y_test)
        scores.append(accuracy)
    
    #save results
    result = [approach, description, channels, metrics, CV, clf_name, np.mean(scores)]
    append_result(result)
    
    # Calculate and print the average accuracy across all folds
    print(f"{clf_name} - Average Accuracy: {round(np.mean(scores),4)}")

df_results

SVM - Average Accuracy: 0.6455
Random Forest - Average Accuracy: 0.6255
AdaBoost - Average Accuracy: 0.5491
XGBoost - Average Accuracy: 0.5655
LightGBM - Average Accuracy: 0.6655


,Feature Extraction,Description,Channels,Metrics,CV,ML Model,Accuracy
0,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
1,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.509091
2,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
3,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
4,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),FP1,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455
5,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,SVM,0.645455
6,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,Random Forest,0.472727
7,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,AdaBoost,0.354545
8,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,XGBoost,0.376364
9,Time-Frequency Domain,PSD Metrics from Epochs (1-50 Hz),All,"Total Power, Relative Power, Peak Frequency, S...",KFold: 5 Folds,LightGBM,0.585455


Save Results
